# Single Samples

Samples of [53812, 15185, 38012, 18694] for visuals from previous notebook. Colab settings to be done

In [ ]:
#%pip install lime shap hf_xet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch
from transformers import pipeline, AutoTokenizer
import lime
from lime.lime_text import LimeTextExplainer
import shap
import os
from typing import List, Union


2025-05-16 19:56:34.533489: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 19:56:34.560927: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-16 19:56:34.560958: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-16 19:56:34.560987: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-16 19:56:34.567691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [6]:
def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True

In [4]:
BATCH_SIZE = 512 

In [5]:
print("Loading models and tokenizers...")
pipe_cardiff = pipeline("text-classification",
                       model="cardiffnlp/twitter-roberta-base-hate-latest",
                       device=0 if torch.cuda.is_available() else -1,
                       batch_size=BATCH_SIZE)
tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

Loading models and tokenizers...


Device set to use cpu


In [7]:
class HateSpeechDataset:
    def __init__(self, texts):
        self.texts = texts
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx]

In [8]:
def batch_predict(texts: Union[List[str], np.ndarray], pipeline_fn):
    """Run predictions in batches for efficiency"""
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    
    dataset = HateSpeechDataset(texts)
    
    # Process in batches
    all_outputs = []
    for i in range(0, len(dataset), BATCH_SIZE):
        batch_texts = [dataset[j] for j in range(i, min(i + BATCH_SIZE, len(dataset)))]
        outputs = pipeline_fn(batch_texts, top_k=2)
        all_outputs.extend(outputs)
    
    # Convert to numpy array with consistent order
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in all_outputs
    ])

In [9]:
def predict_cardiff(texts):
    return batch_predict(texts, pipe_cardiff)

In [ ]:
def generate_lime_visualization(sample_id, text, seed, evaluation):
    """Generate LIME visualization for a given sample and seed, and save as SVG."""
    print(f"Generating {evaluation} LIME visualization for sample {sample_id} with seed {seed}...")
    
    # Set seed for reproducibility
    set_seed(seed)
    
    # Initialize LIME explainer with the specified seed
    explainer = LimeTextExplainer(random_state=seed)
    
    # Generate explanation
    exp = explainer.explain_instance(
        text, 
        predict_cardiff
    )
    
    exp.show_in_notebook(text=True)



In [ ]:
def generate_shap_visualization(sample_id, text, seed, evaluation):
    text_list = [text]
    set_seed(seed)

    explainer = shap.Explainer(predict_cardiff, 
                                masker=shap.maskers.Text(tokenizer_cardiff), 
                                seed=seed)
    shap_values = explainer(text_list)
    print(f"Generating {evaluation} SHAP visualization for sample {sample_id} with seed {seed}...")
    shap.plots.text(shap_values[0])

In [11]:
import pandas as pd
import requests
from io import BytesIO

# GitHub raw content URL for the pickle file
github_raw_url = "https://github.com/Takosaga/master_thesis/raw/main/data/processed/experiement_samples.pkl"
# Download the pickle file
response = requests.get(github_raw_url)

# Check if the request was successful
if response.status_code == 200:
    # Load the pickle data into a pandas DataFrame
    df = pd.read_pickle(BytesIO(response.content))
    print(f"DataFrame loaded successfully with {len(df)} rows")

    # Display the first few rows
    print("\nPreview of the DataFrame:")
    display(df.head())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
    print(f"Response: {response.text}")

DataFrame loaded successfully with 652 rows

Preview of the DataFrame:


,text,extracted_target,platform,is_hatespeech,id,cardiff_score,cardiff_hatespeech
0,"India is one nasty, dirty, filthy, germsy, law...",{nationality},youtube,1,53513,0.994301,1
1,solo que seras bombardeas y destruidos como lo...,{religion},gab,1,8335,0.957913,1
2,@user @user You cannot argue with a retard jer...,"{disability, nationality}",twitter,1,64112,0.994864,1
3,Can I just pound that pussy an pull your hair....,{gender},reddit,1,22034,0.977588,1
4,you do not live in america nazi faggot but you...,{sexuality},gab,1,18143,0.985650,1


In [12]:
text_TN = df.loc[df['id'] == 53812, 'text'].values[0]
text_FN = df.loc[df['id'] == 15185, 'text'].values[0]
text_FP = df.loc[df['id'] == 38012, 'text'].values[0]
text_TP = df.loc[df['id'] == 18694, 'text'].values[0]

In [13]:
id_used = [53812, 15185, 38012, 18694]

In [14]:
seeds_TN = [83811, 97197]
seeds_FN = [83811, 97197]
seeds_FP = [3279, 83811]
seeds_TP = [3279, 14593]

### LIME Visuals

In [ ]:
generate_lime_visualization(id_used[0], text_TN, seeds_TN[0], 'TN' )

In [ ]:
generate_lime_visualization(id_used[0], text_TN, seeds_TN[1], 'TN' )

In [ ]:
generate_lime_visualization(id_used[1], text_FN, seeds_FN[0], 'FN' )


In [ ]:
generate_lime_visualization(id_used[1], text_FN, seeds_FN[1], 'FN' )

In [ ]:
generate_lime_visualization(id_used[2], text_FP, seeds_FP[0], 'FP' )


In [ ]:
generate_lime_visualization(id_used[2], text_FP, seeds_FP[1], 'FP' )

In [ ]:
generate_lime_visualization(id_used[3], text_TP, seeds_TP[0], 'TP' )

In [ ]:

generate_lime_visualization(id_used[3], text_TP, seeds_TP[1], 'TP' )

### SHAP Visuals

In [ ]:
generate_shap_visualization(id_used[0], text_TN, seeds_TN[0], 'TN' )

In [ ]:
generate_shap_visualization(id_used[0], text_TN, seeds_TN[1], 'TN' )

In [ ]:
generate_shap_visualization(id_used[1], text_FN, seeds_FN[0], 'FN' )


In [ ]:
generate_shap_visualization(id_used[1], text_FN, seeds_FN[1], 'FN' )

In [ ]:
generate_shap_visualization(id_used[2], text_FP, seeds_FP[0], 'FP' )

In [ ]:
generate_shap_visualization(id_used[2], text_FN, seeds_FN[1], 'FN' )

In [ ]:
generate_shap_visualization(id_used[3], text_TP, seeds_TP[0], 'TP' )


In [ ]:
generate_shap_visualization(id_used[3], text_TP, seeds_TP[1], 'TP' )